In [1]:
import os
import zipfile
import tensorflow as tf


In [2]:
local = '/home/xia/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local,'r')
zip_ref.extractall('/home/xia/horse-or-human')
zip_ref.close()

In [3]:
train_horse = os.path.join('/home/xia/horse-or-human/train/horses')
train_human = os.path.join('/home/xia/horse-or-human/train/humans')
val_horse = os.path.join('/home/xia/horse-or-human/validation/horses')
val_human = os.path.join('/home/xia/horse-or-human/validation/humans')

In [4]:
my_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3,),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [5]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [6]:
from tensorflow.keras.optimizers import RMSprop
my_model.compile(loss='binary_crossentropy',
             optimizers=RMSprop(lr=0.001),
             metrics=['acc'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1/255)
val_data = ImageDataGenerator(rescale=1/255)

train_img = train_data.flow_from_directory('/home/xia/horse-or-human/train/',
target_size=(150,150),
batch_size = 128, class_mode='binary')

val_img = val_data.flow_from_directory('/home/xia/horse-or-human/validation/',
                                      target_size=(150,150),
                                      batch_size = 32,
                                       class_mode = 'binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [8]:
history = my_model.fit_generator(
train_img,
steps_per_epoch=8,
epochs=15,
verbose=1,
validation_data = val_img,
validation_steps = 8)

Epoch 1/15
8/8 [==============================] - 32s 4s/step - loss: 1.2890 - acc: 0.5973 - val_loss: 1.8401 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 32s 4s/step - loss: 0.5997 - acc: 0.6963 - val_loss: 0.5303 - val_acc: 0.8438
Epoch 3/15
8/8 [==============================] - 27s 3s/step - loss: 0.2945 - acc: 0.8773 - val_loss: 2.7184 - val_acc: 0.5742
Epoch 4/15
8/8 [==============================] - 33s 4s/step - loss: 0.2058 - acc: 0.9131 - val_loss: 1.2451 - val_acc: 0.8164
Epoch 5/15
8/8 [==============================] - 28s 4s/step - loss: 0.1859 - acc: 0.9432 - val_loss: 1.7366 - val_acc: 0.7422
Epoch 6/15
8/8 [==============================] - 37s 5s/step - loss: 0.0634 - acc: 0.9775 - val_loss: 2.0090 - val_acc: 0.7188
Epoch 7/15
8/8 [==============================] - 26s 3s/step - loss: 0.1946 - acc: 0.9341 - val_loss: 0.8064 - val_acc: 0.8984
Epoch 8/15
8/8 [==============================] - 36s 5s/step - loss: 0.2677 - acc: 0.9297 - val_loss: 1

In [18]:
import numpy as np
from ipywidgets import FileUpload
from keras.preprocessing import image

upload = FileUpload(accept='',multiple=False)

  # predicting images
path = '/home/xia/images.jpeg'
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = my_model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(" is a human")
else:
    print(" is a horse")

[1.]
 is a human
